In [1]:
# ppf of cauchy
x = qcauchy(runif(10000000),loc=5,scale=2)
1/mean(dnorm(x,mean=5.387142857142857,sd=sqrt(9/7)))

[1] 7.84634

In [2]:
# ppf of norm
x = qnorm(runif(10000000),mean=5.387142857142857,sd=sqrt(9/7))
1/mean(dcauchy(x,loc=5,scale=2))

[1] 7.846282